


# **GHISA Visualization Application**

Use this application to visualize GHISA spectral data available to download from LP DAAC.
\
\
**Links to LP DAAC & GHISA Resources:**

GHISA CONUS: https://lpdaac.usgs.gov/products/ghisaconusv001/

GHISA Central Asia: https://lpdaac.usgs.gov/products/ghisacasiav001/

GHISA Main Website: https://www.usgs.gov/centers/western-geographic-science-center/science/global-hyperspectral-imaging-spectroscopy



In [ ]:
# if on Google CoLab, install packages into base environment 
import os
if 'COLAB_GPU' in list(os.environ):

    #@title 1. First Time Requirement Installation
    #@markdown Downloads and installs required packages. Click 'Restart Runtime' once complete. (~1 minute)
    #First time setup - Installs all required components
    !pip install geemap &> /dev/null
    !pip install ipyleaflet &> /dev/null
    !pip install ipywidgets &> /dev/null
    !pip install ipympl &> /dev/null
    !pip install npm &> /dev/null
    !pip install hvplot &> /dev/null
    !pip uninstall tornado --yes &> /dev/null
    !pip install tornado

    # RESTART KERNEL AFTER THIS CODE HAS BEEN RUN

In [ ]:
#@title 2. Import Dependencies
#@markdown Installs and activates required packages. (~5 seconds)
#Import dependencies
import pandas as pd
import numpy as np
import geopy
#import geemap
import hvplot.pandas
import panel as pn
import holoviews as hv

In [ ]:
from bokeh.models import SingleIntervalTicker, LinearAxis
from holoviews import opts
hv.extension('bokeh')
#from pygeocoder import Geocoder
#from IPython.display import display

#Cleanup data for mapping. (Map does not need spectrum data)
#Map_DF = pd.read_csv(r'GHISA-USA-EO-1-Hyperion-data-of-crops-1h_gaezs.csv')
#Map_DF = Map_DF.drop(Map_DF.iloc[:, 12:-2], axis=1)

In [ ]:
#@title 3. Create Widget Panel
#@markdown Generates a panel to filter data by Date, Agro-Ecological Zone (AEZ), Study Area, Country, Crop Type, Growth Stage, and Sensor Type.

if 'COLAB_GPU' in list(os.environ):
    pn.extension(comms='colab')

Combined_Crop_Select = pn.widgets.Select(name='Crop:', 
                                         options=['All', 'Alfalfa', 'Cotton', 'Corn', 
                                                  'Rice', 'Soybean', 'Wheat', 'Winter Wheat'],
                                        value='Corn')
Combined_Crop_Select

Combined_Stage_Select = pn.widgets.Select(
    name='Growth Stage:', 
    options=['All', 'Emerge_VEarly', 'Early_Mid', 'Mature_Senesc', 'Late', 'Harvest', 'Critical']
)

Year_Range_Slider = pn.widgets.RangeSlider(
    name='Select Year(s)', start=2006, end=2015, step=1)

Month_Range_Slider = pn.widgets.RangeSlider(
    name='Select Month(s)', start=1, end=12, step=1)

Day_Range_Slider = pn.widgets.RangeSlider(
    name='Select Julian Day(s)', start=1, end=365, step=1)

AEZ_Global_Select = pn.widgets.IntSlider(name='AEZ (Global, 0 for all)', start=0, end=74)

Country_Select = pn.widgets.Select(name='Country:', 
                                   options=['All', 'USA', 'Uzbekistan'], value='Uzbekistan')

State_Select = pn.widgets.Select(
    name='Study Area:', 
    options=['All', 'Arizona', 'California', 'Indiana', 'Kansas', 'Oklahoma', 'Galaba', 
             'South Dakota', 'Texas', 'U.Yusupova', 'Wisconsin']
)


Sensor_Select = pn.widgets.Select(name='Sensor:', options=['All', 'Hyperion', 'ASD Spectroradiometer'], 
                                  value='ASD Spectroradiometer')


pn.Row(pn.WidgetBox(Sensor_Select, Combined_Crop_Select, Combined_Stage_Select), 
       pn.WidgetBox(Year_Range_Slider, Month_Range_Slider, Day_Range_Slider),
       pn.WidgetBox(AEZ_Global_Select, Country_Select, State_Select)
)

In [ ]:
#@title 4. Generate Dataframe from User Selection
#@markdown Creates a dataframe and populates it with data only fitting user selection.

#@markdown Selected data will be exported as a CSV file which can be downloaded. To access the CSV file, go to the menu on the left hand of the screen, select 'Files', and double click 'selected_data' to download.
# Open each data file and assign to a Data Frame

new_df = pd.read_csv(r'https://raw.githubusercontent.com/rmccormick-contractor/Hyperion-ASD-Visualization/main/Croplands_DataViz/asd_long.csv', 
                     low_memory=False, index_col=False, na_values='NaN')
new_df2 = pd.read_csv(r'https://raw.githubusercontent.com/rmccormick-contractor/Hyperion-ASD-Visualization/main/Croplands_DataViz/hyp1_long.csv',
                      low_memory=False, index_col=False, na_values='NaN')
new_df2_1 = pd.read_csv(r'https://raw.githubusercontent.com/rmccormick-contractor/Hyperion-ASD-Visualization/main/Croplands_DataViz/hyp2_1_long.csv',
                        low_memory=False, index_col=False, na_values='NaN')
new_df3 = pd.read_csv(r'https://raw.githubusercontent.com/rmccormick-contractor/Hyperion-ASD-Visualization/main/Croplands_DataViz/hyp2_long.csv',
                      low_memory=False, index_col=False, na_values='NaN')
new_df4 = pd.read_csv(r'https://raw.githubusercontent.com/rmccormick-contractor/Hyperion-ASD-Visualization/main/Croplands_DataViz/hyp3_long.csv',
                      low_memory=False, index_col=False, na_values='NaN')
new_df5 = pd.read_csv(r'https://raw.githubusercontent.com/rmccormick-contractor/Hyperion-ASD-Visualization/main/Croplands_DataViz/hyp4_long.csv',
                      low_memory=False, index_col=False, na_values='NaN')
new_df6 = pd.read_csv(r'https://raw.githubusercontent.com/rmccormick-contractor/Hyperion-ASD-Visualization/main/Croplands_DataViz/hyp5_long.csv',
                      low_memory=False, index_col=False, na_values='NaN')

# Concat all dataframes into one single
new_df = pd.concat([new_df, new_df2])
new_df = pd.concat([new_df, new_df2_1])
new_df = pd.concat([new_df, new_df3])
new_df = pd.concat([new_df, new_df4])
new_df = pd.concat([new_df, new_df5])
new_df = pd.concat([new_df, new_df6])

# Drop unnessary index column
#new_df = new_df.drop(new_df.iloc[:, 0:0], axis=1)

new_df = new_df.convert_dtypes()

# Filter on Crop & Growth Stage
if (Combined_Crop_Select.value != 'All'):
    if (Combined_Crop_Select.value == 'Winter Wheat'):
        new_df = new_df.loc[(new_df['Crop'].values == ('Winter Wheat' or 'Wheat'))]
    else:
        new_df = new_df.loc[(new_df['Crop'].values == Combined_Crop_Select.value)]
if (Combined_Stage_Select.value != 'All'):
    new_df = new_df.loc[(new_df['Stage'].values == Combined_Stage_Select.value)]
   
# Filter based on Year range
new_df = new_df.loc[(new_df['Year'].values >= Year_Range_Slider.value[0])]
new_df = new_df.loc[(new_df['Year'].values <= Year_Range_Slider.value[1])]

# Filter based on Month range
new_df = new_df.loc[(new_df['Month'].values >= Month_Range_Slider.value[0])]
new_df = new_df.loc[(new_df['Month'].values <= Month_Range_Slider.value[1])]

# Filter based on Julian Day range
new_df = new_df.loc[(new_df['JulianDay'].values >= Day_Range_Slider.value[0])]
new_df = new_df.loc[(new_df['JulianDay'].values <= Day_Range_Slider.value[1])]

if (AEZ_Global_Select.value != 0):
    new_df["Global_AEZ"] = pd.to_numeric(new_df["Global_AEZ"], errors='coerce').astype('int')
    new_df = new_df.loc[(new_df['Global_AEZ'].values == AEZ_Global_Select.value)]
    
if (Sensor_Select.value != 'All'):
    new_df = new_df.loc[(new_df['Sensor'].values == Sensor_Select.value)]
    
if (State_Select.value != 'All'):
    new_df = new_df.loc[(new_df['StudyArea'].values.astype(str) == State_Select.value)]
    
if (Country_Select.value != 'All'):
    new_df = new_df.loc[(new_df['Country'].values == Country_Select.value)]


#new_df = new_df.replace(0.0, 'NaN', regex=True)
#new_df = new_df.set_index('MasterID')
##new_df = new_df.drop(columns=['Sensor', 'UniqueID', 'PlotID', 'StudyArea', 'Country', 'USA_AEZ', 'Global_AEZ', 'FID', 'Image', 'Month', 'Year', 'JulianDay', 'Long', 'Lat', 'Stage'])
#new_df = new_df.transpose()
#new_df["Reflectance"] = pd.to_numeric(new_df["Reflectance"], errors='coerce')

#new_df = new_df.reset_index()

#new_df = new_df.dropna()
#new_df = new_df.drop(new_df.iloc[0], axis=1)

#new_df = new_df.pivot_table(
#    index=["MasterID","Crop","Wavelength"],
#    values='Reflectance'
#)

new_df.to_csv('selected_data')
hv.extension('bokeh')

#new_df['Reflectance'].replace('', np.nan)

new_df = new_df.dropna(subset=['Reflectance'])

if (new_df.empty):
    print("No data found for this query. Please expand search parameters.")

In [ ]:
#@title 5. Generate Plot from Dataframe
#@markdown Creates a visual plot of the spectral profiles in the selected data.
hv.extension('bokeh')
new_df = new_df.sort_values(by='Wavelength')
reflectance_plot = new_df.hvplot.line(x = "Wavelength",y = "Reflectance", by="MasterID", legend=False).opts(fontsize={'title': 16, 'labels': 14, 'yticks': 12},xrotation=45, xticks=15)

reflectance_plot

In [ ]:
#@title 6. Generate Mean Plot
#@markdown Creates plots of spectral averages by crop type from the selected data. Generates seperate plots for each sensor.

#@markdown **Averages from broad queries may be less informative.
hv.extension('bokeh')

#Adds new category to ease plotting
test_df = new_df
test_df['crop-sensor'] = test_df[['Crop','Sensor']].agg(' - '.join, axis=1)

colors_dic =  {
    "Alfalfa" : "violet",
    "Corn" : "gold",
    "Cotton": "red",
    "Rice": 'cyan',
    "Soybean": 'forestgreen',
    "Wheat": 'brown',
    "Winter Wheat": "tan"
}

asd_df = test_df.loc[(test_df['Sensor'].values == "ASD Spectroradiometer")]
hyperion_df = test_df.loc[(test_df['Sensor'].values == "Hyperion")]

if (asd_df.empty == False):
    test = asd_df['Crop'].unique()
    test = sorted(test)
    asd_color_list = list()
    for crop in test:
        asd_color_list.append(colors_dic[crop])

if (hyperion_df.empty == False):
    test = hyperion_df['Crop'].unique()
    test = sorted(test)
    hyp_color_list = list()
    for crop in test:
        hyp_color_list.append(colors_dic[crop])

asd_df = asd_df.groupby(['Crop', 'Wavelength'])['Reflectance'].mean()
asd_df = asd_df.dropna()

unique_selected_sensors = list()

if (asd_df.empty == False):
    asd_plot = asd_df.hvplot.line(x = "Wavelength",
                                  y = "Reflectance",
                                  by='Crop', 
                                  title='Average Reflectance (ASD Spectroradiometer)', 
                                  ylabel='Reflectance (%)',
                                  xlabel='Wavelength (nm)',
                                  line_color = asd_color_list).opts(fontsize={'title': 16, 
                                                                          'labels': 14, 
                                                                          'yticks': 12},
                                                                xrotation=45,
                                                                xticks=15)
    unique_selected_sensors.append(asd_plot)

hyperion_df = hyperion_df.groupby(['Crop', 'Wavelength'])['Reflectance'].mean()
hyperion_df = hyperion_df.dropna()

if (hyperion_df.empty == False):
    hyperion_plot = hyperion_df.hvplot.line(x = "Wavelength",y = "Reflectance",by=['Crop'], title='Average Reflectance (EO1-Hyperion)', ylabel='Reflectance (%)', xlabel='Wavelength (nm)', line_color=hyp_color_list).opts(fontsize={'title': 16, 'labels': 14, 'yticks': 12},xrotation=45, xticks=15)
    unique_selected_sensors.append(hyperion_plot)

if (asd_df.empty):
    asd_plot = asd_df.hvplot.line(x = "Wavelength",y = "Reflectance",by='Crop', title='Average Reflectance (ASD Spectroradiometer)', ylabel='Reflectance (%)',xlabel='Wavelength (nm)').opts(fontsize={'title': 16, 'labels': 14, 'yticks': 12},xrotation=45,xticks=15)
if (hyperion_df.empty):
    hyperion_plot = hyperion_df.hvplot.line(x = "Wavelength",y = "Reflectance",by=['Crop'], title='Average Reflectance (EO1-Hyperion)', ylabel='Reflectance (%)', xlabel='Wavelength (nm)').opts(fontsize={'title': 16, 'labels': 14, 'yticks': 12},xrotation=45, xticks=15)

hyperion_plot+asd_plot

In [ ]:
#@title 7. Compare Sensor Averages
#@markdown Directly compare average reflectance of one crop type from different sensors.
hv.extension('bokeh')

#test_df['crop-sensor'] = test_df[['Crop','Sensor']].agg(' - '.join, axis=1)
mean_df = test_df
mean_df = mean_df.groupby(['Crop', 'Wavelength', 'Sensor'])['Reflectance'].mean()
mean_df = mean_df.dropna()


mean_plot = mean_df.hvplot.line(x = "Wavelength",y = "Reflectance",by='Sensor', groupby='Crop', widget_location='top_right', title='Average Reflectance', ylabel='Reflectance (%)', xlabel='Wavelength (nm)')
mean_plot
#.opts(fontsize={'title': 16, 'labels': 14, 'yticks': 12},xrotation=45, xticks=15)
